# Metadata

**L1 Taxonomy** - Security

**L2 Taxonomy** - Authentication

**Subtopic** - Ensuring logouts truly invalidate sessions (e.g.server tracks and invalidates JWT or session tokens)

**Use Case** - Develop a Python script that generates and validates JWT tokens. The script should include a function to invalidate these tokens, simulating a user logout. Use the 'PyJWT' library to handle JWT operations, ensuring that the token invalidation function truly makes the tokens unusable.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
PyJWT==2.10.1
```


# Prompt
Problem Statement:
- Develop a Python script that generates and validates JWT tokens.
- The script should include a function to invalidate these tokens, simulating a user logout.
- Use the 'PyJWT' library to handle JWT operations, ensuring that the token invalidation function truly makes the tokens unusable.
- Please make sure not to include unnecessary explanations or extra details. Just provide the complete implementation from start to end as per the prompt.

Input Format:
- A dictionary `payload` containing user data (e.g., {"user_id": 123}).
- A string `secret` used as the signing key for generating and validating the JWT.
- An optional integer `expiry_seconds` specifying token validity duration in seconds (default is 3600).
- A string `token` representing the JWT to be validated or invalidated.

Input Constraints:
- `payload` must be a non-empty dictionary containing serializable key-value pairs.
- `secret` must be a non-empty string.
- `expiry_seconds`, if provided, must be a positive integer (e.g., > 0).
- `token` must be a valid JWT string (three base64url-encoded segments separated by dots).
- Token size must not exceed standard HTTP header limits (~8KB recommended max).

Output Format:
- A string representing the generated JWT token when calling the token generation function.
- A dictionary containing the decoded payload when validating a valid token.
- An exception or error message if the token is invalid, expired, or blacklisted.
- No return value for the invalidate function, but the token is added to a blacklist internally.

Example:
```python
Example:
Input:
- payload = {"user_id": 101, "role": "admin"}
- secret = "mysecretkey"
- expiry_seconds = 3600

Process:
- Call generate_token(payload, expiry_seconds)
- Use the returned token to call validate_token(token)
- Optionally, call invalidate_token(token) to simulate logout

Output:
- Generated Token: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...
- Decoded Payload after validation: {"user_id": 101, "role": "admin", "exp": <timestamp>}
- After invalidation: Exception raised if validate_token(token) is called again
```

Class Definition:
```python
- class JWTManager:
    - Manages JWT generation, validation, and invalidation using PyJWT.

Function Signatures:
- def __init__(self, secret: str):
    - Initializes the JWTManager with a secret key.

- def generate_token(self, payload: dict, expiry_seconds: int = 3600) -> str:
    - Generates a JWT token with the given payload and expiry.

- def validate_token(self, token: str) -> dict:
    - Validates the JWT token and returns the decoded payload if valid.

- def invalidate_token(self, token: str) -> None:
    - Invalidates the given JWT token by adding it to a blacklist.
```

# Requirements
Explicit Requirements:
- Use the `PyJWT` library to handle JWT encoding and decoding.
- Implement token generation with customizable expiration.
- Implement token validation that checks signature and expiry.
- Provide a function to invalidate tokens (simulate logout).
- Invalidated tokens must not pass future validation.

Implicit Requirements:
- Use an in-memory structure (e.g., set or dict) to track invalidated tokens.
- Expired tokens should be rejected during validation.
- Tokens must be securely signed using the provided `secret` key.
- Token generation should include standard claims like `exp` (expiration).
- generate_token must add the exp claim to the payload internally.
- Handle common exceptions (e.g., invalid signature, expired token) gracefully.
- All input data (payload, secret) must be validated for type and structure.

Solution Expectations:
- A class-based Python implementation named `JWTManager` using the `PyJWT` library.
- Methods to:
  - Generate a JWT token with a payload and expiration.
  - Validate a token and return its payload if valid.
  - Invalidate a token and ensure it cannot be used again.
- Use of an internal blacklist (e.g., a set) to store invalidated tokens.
- Proper error handling for expired, tampered, or blacklisted tokens.
- Secure signing and decoding using HMAC with SHA-256.
- All code must follow PEP8 and PEP257 standards.
- Include at least one working example demonstrating all three operations.
- Optional: Unit tests covering valid, expired, and blacklisted tokens.

Edge Cases and Behavior:
- Empty Payload:
  - Behavior: Token generation should raise a ValueError.

- Missing or Empty Secret Key:
  - Behavior: Raise an error during token generation or validation to enforce secure signing.

- Expired Token:
  - Behavior: Validation should fail with an appropriate error (e.g., ExpiredSignatureError).

- Invalid Token Format (e.g., missing segments or corrupted base64):
  - Behavior: Raise a decode error (e.g., InvalidTokenError) and return a user-friendly message.

- Blacklisted Token:
  - Behavior: Even if signature and expiry are valid, validation should fail if the token is blacklisted.

- Tampered Token (e.g., payload or signature altered):
  - Behavior: Validation must fail with a signature verification error.

- Duplicate Invalidation:
  - Behavior: Repeated calls to invalidate the same token should be idempotent (no error thrown).

- Short Expiry (e.g., 1 second):
  - Behavior: Token should become unusable immediately after expiry; validation must reflect this.

- Very Long Payloads:
  - Behavior: Validate payload size to avoid HTTP header size limits (typically 8KB); raise error if exceeded.

- Clock Skew:
  - Behavior: Optional: Use leeway in decoding to handle small time differences (e.g., `jwt.decode(..., leeway=5)`).

Solution Constraints:
- Must use only the `PyJWT` library for JWT encoding and decoding.
- Must use HMAC with SHA-256 (`HS256`) as the signing algorithm.
- Must implement token invalidation via an in-memory blacklist (e.g., `set` or `dict`).
- Token validation must fail for blacklisted or expired tokens.
- Must not persist tokens or blacklists to disk or database (in-memory only).
- All tokens must include an `exp` (expiration) claim.
- Secret key must be provided at initialization and consistently used across methods.
- Code must handle all JWT-related exceptions gracefully (e.g., expired, invalid signature, malformed).
- No external dependencies beyond `PyJWT` and standard library.
- The blacklist must be implemented as a class-level or instance-level in-memory set, not global or static.
- The `generate_token` method must not modify the original input `payload` dictionary.
- Token validation must fail immediately without attempting decoding if the token is already blacklisted.
- The system must not expose or return the secret key in any public method or print/debug output.

In [ ]:
# code
"""JWT utility module for generation, validation, and blacklisting."""

import jwt
import time
from datetime import datetime, timezone


class JWTManager:
    """Manage JWT generation, validation, and invalidation using PyJWT."""

    MAX_TOKEN_BYTES = 8192

    def __init__(self, secret: str):
        """
        Initialize the JWTManager with a secret key.

        Args:
            secret: The secret key used for signing and verifying JWTs.

        Raises:
            ValueError: If the secret is empty.
        """
        if not secret:
            raise ValueError("Secret key cannot be empty.")
        self.__secret = secret
        self.__blacklist = set()
        self.__algorithm = "HS256"

    def generate_token(self, payload: dict, expiry_seconds: int = 3600) -> str:
        """
        Generate a JWT token with the given payload and expiry.

        Args:
            payload: A dictionary containing user data.
            expiry_seconds: The duration in seconds until the token expires.

        Returns:
            A string representing the generated JWT token.

        Raises:
            ValueError: If the payload is invalid or too large.
        """
        if not payload or not isinstance(payload, dict):
            raise ValueError("Payload must be a non-empty dictionary.")
        if not isinstance(expiry_seconds, int) or expiry_seconds <= 0:
            raise ValueError("Expiry seconds must be a positive integer.")

        token_payload = payload.copy()
        token_payload["exp"] = int(
            datetime.now(timezone.utc).timestamp() + expiry_seconds
        )

        try:
            token = jwt.encode(
                token_payload,
                self.__secret,
                algorithm=self.__algorithm
            )
            token_bytes = token.encode("utf-8")
            if len(token_bytes) > self.MAX_TOKEN_BYTES:
                raise ValueError(
                    f"Generated token byte size ({len(token_bytes)} bytes) "
                    f"exceeds recommended limit"
                    f" ({self.MAX_TOKEN_BYTES} bytes)."
                )
            return token
        except Exception as e:
            raise ValueError(f"Error generating token: {e}")

    def validate_token(self, token: str) -> dict:
        """
        Validate the JWT token and return the decoded payload if valid.

        Args:
            token: The JWT string to be validated.

        Returns:
            A dictionary containing the decoded payload.

        Raises:
            jwt.ExpiredSignatureError: If the token has expired.
            jwt.InvalidSignatureError: If the token's signature is invalid.
            jwt.DecodeError: If the token is malformed or invalid.
            ValueError: If the token is blacklisted.
        """
        if not isinstance(token, str) or token.count('.') != 2:
            raise jwt.DecodeError(
                "Invalid token format: Must have three segments."
            )

        if token in self.__blacklist:
            raise ValueError("Token has been invalidated (blacklisted).")

        try:
            return jwt.decode(
                token,
                self.__secret,
                algorithms=[self.__algorithm],
                leeway=5
            )
        except jwt.ExpiredSignatureError:
            raise jwt.ExpiredSignatureError("Token has expired.")
        except jwt.InvalidSignatureError:
            raise jwt.InvalidSignatureError("Invalid token signature.")
        except jwt.DecodeError as e:
            raise jwt.DecodeError(f"Invalid token: {e}")
        except Exception as e:
            raise ValueError(f"Unexpected error during validation: {e}")

    def invalidate_token(self, token: str) -> None:
        """
        Invalidate the given JWT token by adding it to a blacklist.

        Args:
            token: The JWT string to be invalidated.
        """
        if token:
            self.__blacklist.add(token)


if __name__ == "__main__":
    secret_key = "supersecretkey123"
    jwt_manager = JWTManager(secret_key)

    payload = {"user_id": 101, "role": "admin"}
    try:
        generated_token = jwt_manager.generate_token(payload,
                                                     expiry_seconds=30)
        print(f"Generated Token:\n{generated_token}")
    except ValueError as e:
        print(f"Error generating token: {e}")
        generated_token = None

    if generated_token:
        try:
            decoded_payload = jwt_manager.validate_token(generated_token)
            print(f"\nDecoded Payload:\n{decoded_payload}")
        except (
            jwt.ExpiredSignatureError,
            jwt.InvalidSignatureError,
            jwt.DecodeError,
            ValueError
        ) as e:
            print(f"Error validating token: {e}")

        print("\nInvalidating the token...")
        jwt_manager.invalidate_token(generated_token)
        print("Token invalidated.")

        try:
            jwt_manager.validate_token(generated_token)
        except (
            jwt.ExpiredSignatureError,
            jwt.InvalidSignatureError,
            jwt.DecodeError,
            ValueError
        ) as e:
            print(f"Expected validation failure after invalidation: {e}")

        print("\nTesting with an expired token...")
        expired_payload = {"user_id": 202}
        expired_token = jwt_manager.generate_token(expired_payload,
                                                   expiry_seconds=1)
        print(f"Generated expired token:\n{expired_token}")
        time.sleep(2)
        try:
            jwt_manager.validate_token(expired_token)
        except (
            jwt.ExpiredSignatureError,
            jwt.InvalidSignatureError,
            jwt.DecodeError,
            ValueError
        ) as e:
            print(f"Expected validation failure for expired token: {e}")

    print("\n--- Edge Cases ---")

    try:
        jwt_manager.generate_token({}, 3600)
    except ValueError as e:
        print(f"Expected error for empty payload: {e}")

    try:
        JWTManager("")
    except ValueError as e:
        print(f"Expected error for empty secret: {e}")

    try:
        jwt_manager.validate_token("invalid.token.format")
    except jwt.DecodeError as e:
        print(f"Expected error for invalid format (incorrect dots): {e}")

    try:
        jwt_manager.validate_token("abc.def.ghi")
    except jwt.DecodeError as e:
        print(f"Expected error for malformed token content: {e}")

    if generated_token:
        tampered_token = generated_token[:-5] + "AAAAA"
        try:
            jwt_manager.validate_token(tampered_token)
        except (
            jwt.ExpiredSignatureError,
            jwt.InvalidSignatureError,
            jwt.DecodeError,
            ValueError
        ) as e:
            print(f"Expected error for tampered token: {e}")

    if generated_token:
        print("\nAttempting duplicate invalidation...")
        jwt_manager.invalidate_token(generated_token)
        print("Duplicate invalidation call processed successfully.")

    long_payload = {"data": "A" * 6000}
    try:
        long_token = jwt_manager.generate_token(long_payload, 3600)
        print(f"Generated long token (length: {len(long_token)}).")
    except ValueError as e:
        print(f"Expected error for oversized token: {e}")


In [ ]:
# tests

"""Test cases for JWTManager class covering all requirements."""

import unittest
import time
import jwt
import json
import base64
from main import JWTManager


class TestJWTManager(unittest.TestCase):
    """Test cases for JWTManager class."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.secret = "test_secret_key_123"
        self.jwt_manager = JWTManager(self.secret)
        self.valid_payload = {"user_id": 123, "username": "testuser"}

    def tearDown(self):
        """Clean up after each test method."""
        if hasattr(self.jwt_manager, "blacklist"):
            self.jwt_manager.blacklist.clear()

    def test_generate_token_valid_payload(self):
        """Test successful token generation with valid payload."""
        token = self.jwt_manager.generate_token(
            self.valid_payload, expiry_seconds=3600
        )

        self.assertIsInstance(token, str)

        parts = token.split(".")
        self.assertEqual(len(parts), 3)

        for part in parts:
            try:
                padding = 4 - (len(part) % 4)
                if padding != 4:
                    part += "=" * padding
                base64.urlsafe_b64decode(part)
            except Exception:
                self.fail(f"Token part is not valid base64url: {part}")

    def test_validate_token_success(self):
        """Test successful token validation."""
        token = self.jwt_manager.generate_token(
            self.valid_payload, expiry_seconds=3600
        )
        decoded_payload = self.jwt_manager.validate_token(token)

        self.assertEqual(
            decoded_payload["user_id"], self.valid_payload["user_id"]
        )
        self.assertEqual(
            decoded_payload["username"], self.valid_payload["username"]
        )

        self.assertIn("exp", decoded_payload)
        self.assertIsInstance(decoded_payload["exp"], int)

    def test_invalidate_token_flow(self):
        """Test token invalidation and subsequent validation failure."""
        token = self.jwt_manager.generate_token(
            {"user_id": 456, "role": "admin"}, expiry_seconds=3600
        )

        result = self.jwt_manager.validate_token(token)
        self.assertEqual(result["user_id"], 456)
        self.assertEqual(result["role"], "admin")

        self.jwt_manager.invalidate_token(token)

        with self.assertRaises(Exception) as context:
            self.jwt_manager.validate_token(token)
        self.assertIn("invalidated", str(context.exception).lower())

    def test_generate_token_empty_payload(self):
        """Test error handling for empty payload."""
        with self.assertRaises(ValueError) as context:
            self.jwt_manager.generate_token({}, expiry_seconds=3600)
        self.assertIn("empty", str(context.exception).lower())

    def test_invalid_secret_key_init(self):
        """Test error handling for invalid secret keys."""
        with self.assertRaises(ValueError) as context:
            JWTManager("")
        self.assertIn("secret", str(context.exception).lower())

        with self.assertRaises(ValueError) as context:
            JWTManager(None)
        self.assertIn("secret", str(context.exception).lower())

    def test_validate_expired_token(self):
        """Test validation of expired tokens."""
        token = self.jwt_manager.generate_token(
            {"user_id": 789, "action": "test"}, expiry_seconds=1
        )

        time.sleep(6)

        with self.assertRaises(jwt.ExpiredSignatureError):
            self.jwt_manager.validate_token(token)

    def test_validate_invalid_token_format(self):
        """Test validation of malformed tokens."""
        invalid_tokens = [
            "not.a.token",
            "only.two",
            "",
            "a.b.c.d",
            "invalid base64 content",
            "eyJhbGci.eyJ1c2Vy.!!invalid!!",
        ]

        for invalid_token in invalid_tokens:
            with self.assertRaises(Exception):
                self.jwt_manager.validate_token(invalid_token)

    def test_validate_tampered_token(self):
        """Test validation of tampered tokens."""
        token = self.jwt_manager.generate_token(
            {"user_id": 999}, expiry_seconds=3600
        )

        parts = token.split(".")
        tampered_payload = parts[1][:-1] + (
            "A" if parts[1][-1] != "A" else "B"
        )
        tampered_token = f"{parts[0]}.{tampered_payload}.{parts[2]}"

        with self.assertRaises(jwt.InvalidSignatureError):
            self.jwt_manager.validate_token(tampered_token)

    def test_duplicate_invalidation(self):
        """Test multiple invalidations of the same token."""
        token = self.jwt_manager.generate_token(
            {"user_id": 111}, expiry_seconds=3600
        )

        self.jwt_manager.invalidate_token(token)
        self.jwt_manager.invalidate_token(token)
        self.jwt_manager.invalidate_token(token)

        with self.assertRaises(Exception):
            self.jwt_manager.validate_token(token)

    def test_very_short_expiry(self):
        """Test tokens with very short expiry times."""
        payload = {"session_id": "abc123", "temp": True}
        token = self.jwt_manager.generate_token(payload, expiry_seconds=1)

        result = self.jwt_manager.validate_token(token)
        self.assertEqual(result["session_id"], "abc123")

        time.sleep(6)

        with self.assertRaises(jwt.ExpiredSignatureError):
            self.jwt_manager.validate_token(token)

    def test_large_payload(self):
        """Test handling of large payload data."""
        large_data = "x" * 7000
        large_payload = {
            "user_id": 222,
            "large_field": large_data,
            "metadata": {"version": 1, "type": "test"},
        }

        try:
            token = self.jwt_manager.generate_token(
                large_payload, expiry_seconds=3600
            )
            decoded = self.jwt_manager.validate_token(token)
            self.assertEqual(decoded["user_id"], 222)
            self.assertEqual(len(decoded["large_field"]), 7000)
        except ValueError as e:
            self.assertIn("size", str(e).lower())

    def test_blacklist_checked_before_decode(self):
        """Test blacklist checking before token decoding."""
        token = self.jwt_manager.generate_token(
            {"user_id": 333}, expiry_seconds=3600
        )

        self.jwt_manager.invalidate_token(token)

        with self.assertRaises(Exception) as context:
            self.jwt_manager.validate_token(token)
        self.assertIn("invalidated", str(context.exception).lower())

    def test_different_secret_keys(self):
        """Test token validation with different secret keys."""
        jwt_manager1 = JWTManager("secret_key_1")
        token = jwt_manager1.generate_token(
            {"user_id": 777}, expiry_seconds=3600
        )

        jwt_manager2 = JWTManager("secret_key_2")
        with self.assertRaises(jwt.InvalidSignatureError):
            jwt_manager2.validate_token(token)

    def test_algorithm_hs256(self):
        """Test token uses HS256 algorithm."""
        token = self.jwt_manager.generate_token(
            {"test": "data"}, expiry_seconds=3600
        )

        header_part = token.split(".")[0]
        padding = 4 - (len(header_part) % 4)
        if padding != 4:
            header_part += "=" * padding

        header = json.loads(base64.urlsafe_b64decode(header_part))
        self.assertEqual(header["alg"], "HS256")

    def test_generate_token_preserves_original_payload(self):
        """Test original payload is not modified during token generation."""
        original_payload = {"user_id": 888, "data": "original"}
        payload_copy = original_payload.copy()

        self.jwt_manager.generate_token(original_payload, expiry_seconds=3600)

        self.assertEqual(original_payload, payload_copy)
        self.assertNotIn("exp", original_payload)

    def test_multiple_users_tokens(self):
        """Test managing tokens for multiple users."""
        tokens = []

        for i in range(5):
            payload = {"user_id": i, "username": f"user_{i}"}
            token = self.jwt_manager.generate_token(
                payload, expiry_seconds=3600
            )
            tokens.append(token)

        self.jwt_manager.invalidate_token(tokens[1])
        self.jwt_manager.invalidate_token(tokens[3])

        for i, token in enumerate(tokens):
            if i in [1, 3]:
                with self.assertRaises(Exception):
                    self.jwt_manager.validate_token(token)
            else:
                result = self.jwt_manager.validate_token(token)
                self.assertEqual(result["user_id"], i)

    def test_invalid_expiry_seconds(self):
        """Test error handling for invalid expiry values."""
        invalid_expiry_values = [0, -1, -100, None]

        for expiry in invalid_expiry_values:
            with self.assertRaises(ValueError):
                self.jwt_manager.generate_token(
                    self.valid_payload, expiry_seconds=expiry
                )

    def test_token_type_validation(self):
        """Test validation with invalid token types."""
        invalid_tokens = [123, None, [], {}, True]

        for invalid_token in invalid_tokens:
            with self.assertRaises(Exception):
                self.jwt_manager.validate_token(invalid_token)

    def test_complete_user_session(self):
        """Test complete user session workflow."""
        login_payload = {
            "user_id": 1001,
            "username": "john_doe",
            "email": "john@example.com",
            "roles": ["user", "premium"],
        }
        session_token = self.jwt_manager.generate_token(
            login_payload, expiry_seconds=7200
        )

        for i in range(3):
            user_data = self.jwt_manager.validate_token(session_token)
            self.assertEqual(user_data["user_id"], 1001)
            self.assertEqual(user_data["username"], "john_doe")
            self.assertIn("exp", user_data)

        self.jwt_manager.invalidate_token(session_token)

        with self.assertRaises(Exception) as context:
            self.jwt_manager.validate_token(session_token)
        self.assertIn("invalidated", str(context.exception).lower())

    def test_payload_data_types(self):
        """Test handling of various payload data types."""
        complex_payload = {
            "user_id": 500,
            "is_active": True,
            "score": 98.5,
            "tags": ["python", "jwt", "security"],
            "metadata": {"created": "2024-01-01", "version": 2},
        }

        token = self.jwt_manager.generate_token(
            complex_payload, expiry_seconds=3600
        )
        decoded = self.jwt_manager.validate_token(token)

        self.assertEqual(decoded["user_id"], 500)
        self.assertEqual(decoded["is_active"], True)
        self.assertEqual(decoded["score"], 98.5)
        self.assertEqual(decoded["tags"], ["python", "jwt", "security"])
        self.assertEqual(decoded["metadata"]["version"], 2)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/a1c61ed9-6fd1-41be-94ab-e896285c7a02?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F81044%2Fview

#### Model code:

```python
# code generated by the model
import time
import jwt
from jwt import ExpiredSignatureError, InvalidSignatureError, DecodeError


class JWTManager:
    """Manages JWT generation, validation, and invalidation using PyJWT."""
    
    def __init__(self, secret: str):
        """Initialize with a secret key."""
        if not secret or not isinstance(secret, str):
            raise ValueError("Secret key must be a non-empty string.")
        self.secret = secret
        self.invalidated_tokens = set()

    def generate_token(self, payload: dict, expiry_seconds: int = 3600) -> str:
        """Generate a JWT token with the given payload and expiry."""
        if not payload or not isinstance(payload, dict):
            raise ValueError("Payload must be a non-empty dictionary.")
        if not isinstance(expiry_seconds, int) or expiry_seconds < 1:
            raise ValueError("Expiry seconds must be a positive integer.")

        exp_time = int(time.time()) + expiry_seconds
        to_encode = payload.copy()
        to_encode["exp"] = exp_time
        
        token = jwt.encode(to_encode, self.secret, algorithm="HS256")
        if len(token) > 8192:
            raise ValueError("Generated token exceeds size limit.")

        return token

    def validate_token(self, token: str) -> dict:
        """Validate the JWT token and return its payload if valid."""
        if token in self.invalidated_tokens:
            raise ValueError("Token has been invalidated.")
        
        # <Issue>: Missing explicit 3-segment token format check.

        if not token or len(token) > 8192:
            raise ValueError("Token is invalid or exceeds size limit.")

        try:
            decoded = jwt.decode(token, self.secret, algorithms=["HS256"])
            return decoded
        except ExpiredSignatureError:
            raise ValueError("Token has expired.")
        except InvalidSignatureError:
            raise ValueError("Invalid token signature.")
        except DecodeError:
            raise ValueError("Token decode error.")
        except Exception as e:
            raise ValueError(f"Token validation error: {str(e)}")

    def invalidate_token(self, token: str) -> None:
        """Invalidate the given JWT token by adding it to a blacklist."""
        self.invalidated_tokens.add(token)


if __name__ == "__main__":
    # Example usage
    manager = JWTManager(secret="mysecretkey")
    payload = {"user_id": 101, "role": "admin"}
    
    # Generate token
    token = manager.generate_token(payload, expiry_seconds=5)
    print("Generated Token:", token)
    
    # Validate token
    decoded_payload = manager.validate_token(token)
    print("Decoded Payload:", decoded_payload)
    
    # Invalidate token
    manager.invalidate_token(token)
    
    # Attempt validation again (will raise an error)
    try:
        manager.validate_token(token)
    except ValueError as e:
        print("Validation after invalidation:", str(e))
```